## Part 3 Molecular adsorption

Supervisor: Prof. Aschauer

Assistants: Dr. Xing Wang

Room:	N216

### 1. Surface
We have two surfaces: stoichiometric and defective surfaces. Here we calculate the stoichiometric surface. Please create a new Jupyter-notebook to calculate the defective one.

In [ ]:
# read the optimized sruface structure from Part 2
from xespresso import Espresso
calc = Espresso(label = 'surface/lto-010')
slab_opt = slab.calc.results['atoms']


### 2. Molecular adsorption
The La$_2$Ti$_2$O$_7$ surface exposes four undercoordinated La and four undercoordinated Ti atoms, surrounded by O atoms, that represent adsorption sites for the NH$_3$. Molecular NH$_3$ will adsorb on the surface in a lots of geometries. We will try the configurations with the N atoms about 1Å above the follwoing surface sites:

* at the top site (La, Ti)
* at the bridge site between La and Ti atoms

Let's look at the La$_2$Ti$_2$O$_7$ surface again and chose approximate positions for the adsorption sites. Then add those position information to the `adsorbate_info` of the slab.

In [29]:
# set adsorption site on the slab
positions = slab.get_scaled_positions()
sites = {
    'Ti-top': positions[74][0:2], # the (x, y) position of the atom 51 in the slab
    # 'O-top': positions[75][0:2], 
    'La-top': positions[67][0:2], 
    'Ti-La-bridge': (positions[67][0:2] + positions[73][0:2])/2.0,  # bridge site between atom 67 and atom 73
    }
slab_opt.info['adsorbate_info'] = {'sites': sites}

Is there other posible adsorption sites and geometries?

Now we add NH$_3$ on the adsorption sites. Please check all configurations.

In [36]:
from ase.build import molecule, add_adsorbate
nh3 = molecule('NH3')
nh3.rotate(180, 'x') # rotate the molecule to make N atoms in the downside
jobs = {}
for site in slab.info['adsorbate_info']['sites']:
    print(site)
    atoms = slab_opt.copy()
    add_adsorbate(atoms, nh3, 1.0, site, mol_index=0)
    jobs[site] = atoms
# We can check the structure 
view(jobs.values())
site = 'Ti-top'
view_x3d_n(jobs[site], bond = 1.0, label = True, output = 'htmls/%s-N.html'%site)

Ti-top
O-top
La-top
Ti-La-bridge



Each calculation will take a long time, therefore we will not run the calculation one by one. Instead, a `mypool` function will be used to sumbit all the calculations at the same time, thus all jobs will run parallelly.

In [15]:
from xespresso import Espresso
from xespresso.tools import mypool
from pseudo import mypseudo


queue = {'nodes': 4, 'ntasks-per-node': 20, 'partition': 'empi', 'mem-per-cpu': '5G', 'time': '23:59:00'}

molecular_energies = {}
# def a function to run a calculation for one site
def run(job, atoms):
    calc = Espresso(label = 'surface/nh3/%s'%job,
                    pseudopotentials = mypsudo,
                    calculation = 'relax',   # allow atoms to move
                    ecutwfc = 40,
                    occupations = 'smearing',
                    degauss = 0.01,
                    kpts = (2, 4, 1),        # k-points, 
                    electron_maxstep = 200,
                    mixing_mode = 'local-TF',
                    nspin = 2,
                    lda_plus_u = True,
                    input_data = {'input_ntyp': input_ntyp},
                    queue = queue,
                    parallel = '-nk 5',)
    atoms.calc = calc
    atoms.get_potential_energy()
    e = atoms.calc.results['energy']
    print('{0} {1:3.3f}'.format(site, e))
    return job, calc
# submit the calculation for all site at the same time
mypool(jobs, run)

Same geometry and parameters, try to check the results are available or not.
top-top -11543.680
Same geometry and parameters, try to check the results are available or not.
top-fcc -11543.638


### 4. Calculation the adsorption energy
The adsorption energy is given by

$\Delta E_{ads} = E_{surf + ads} - (E_{surf} + E_{ads})$

where $E_{surf + ads}$ is the energy of the surface with the adsorbate (NH$_3$), $E_{surf}$ is the energy of the clean surface and $E_{ads}$ is the energy of the adsorbate. A negative value means that there is an energy gain for adsorption, while a positive value means that adsorption is not energetically favorable.

In [40]:
#=====================================================
#  Molecular adsorption
E_nh3 = ?
E_lto = ?
molecular_energies = {
'Ti-top': ?,
'O-top': ?,
'La-top': ?,
?
?
}
print('Sites     Adsorption energies (eV)')
for site, E_ads in molecular_energies.items():
    dE_ads = E_ads - (E_lto + E_nh3)
    print('{0:10s}  {1:1.2f}'.format(site, dE_ads))

Sites     Adsorption energies (eV)
top-top     -0.74
top-fcc     -0.70
